# Diseño con la cuadricula

In [5]:
import pygame

# Configuración de la ventana
window_width = 1200
window_height = 800

# Configuración de la cuadrícula
grid_size = 10
grid_color = (200, 200, 200)  # Color de la cuadrícula en formato RGB

# Configuración de los elementos adicionales
road_width = 200
sidewalk_width = 100
green_color = (0, 255, 0)
white_color = (255, 255, 255)
grey_color = (128, 128, 128)
yellow_color = (255, 255, 0)

# Inicializar pygame y la ventana
pygame.init()
window = pygame.display.set_mode((window_width, window_height))
pygame.display.set_caption("Cuadrícula con elementos adicionales")

def draw_grid():
    # Dibujar líneas verticales
    for x in range(0, window_width, grid_size):
        pygame.draw.line(window, grid_color, (x, 0), (x, window_height))

    # Dibujar líneas horizontales
    for y in range(0, window_height, grid_size):
        pygame.draw.line(window, grid_color, (0, y), (window_width, y))

def draw_elements():
    # Dibujar los elementos adicionales
    pygame.draw.rect(window, white_color, (window_width/2 - road_width/2 - sidewalk_width, 0, sidewalk_width, window_height))
    pygame.draw.rect(window, white_color, (window_width/2 + road_width/2, 0, sidewalk_width, window_height))
    pygame.draw.rect(window, white_color, (0, window_height/2 - road_width/2 - sidewalk_width, window_width, sidewalk_width))
    pygame.draw.rect(window, white_color, (0, window_height/2 + road_width/2, window_width, sidewalk_width))

    pygame.draw.rect(window, grey_color, (window_width/2 - road_width/2, 0, road_width, window_height))
    pygame.draw.rect(window, grey_color, (0, window_height/2 - road_width/2, window_width, road_width))

    for y in range(0, window_height, 80):
        pygame.draw.rect(window, yellow_color, (window_width/2 - 5, y, 10, 40))
    for x in range(0, window_width, 80):
        pygame.draw.rect(window, yellow_color, (x, window_height/2 - 5, 40, 10))

# Bucle principal del programa
running = True
while running:
    # Manejo de eventos
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Obtener las coordenadas del cursor relativas a la cuadrícula
    mouse_x, mouse_y = pygame.mouse.get_pos()
    grid_x = (mouse_x // grid_size) * grid_size
    grid_y = (mouse_y // grid_size) * grid_size

    # Rellenar la ventana con color verde
    window.fill(green_color)

    # Dibujar los elementos adicionales
    draw_elements()

    # Dibujar la cuadrícula
    draw_grid()

    # Mostrar las coordenadas del cursor en la cuadrícula
    font = pygame.font.Font(None, 30)
    text = font.render(f"({grid_x}, {grid_y})", True, white_color)
    window.blit(text, (10, 10))

    # Actualizar la ventana
    pygame.display.flip()

# Cerrar pygame al salir
pygame.quit()


# PRUEBAS DE COLISION

In [3]:
import pygame

# Constantes
WIDTH = 800
HEIGHT = 600
CAR_SPEED = 1
FPS = 30

# Inicialización de Pygame
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()

# Definición de colores
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255, 20)

# Clase Carro
class Carro:
    def __init__(self, x, y, width, height, color, direction):
        self.rect = pygame.Rect(x, y, width, height)
        self.color = color
        self.direction = direction
        self.has_collision = False

    def move(self, semaforo, carros):
        if semaforo.is_red() and self.has_collision:
            return  # No hacer nada si hay una colisión y el semáforo está en rojo

        if self.direction == 'up':
            self.rect.y -= CAR_SPEED
        elif self.direction == 'down':
            self.rect.y += CAR_SPEED
        elif self.direction == 'left':
            self.rect.x -= CAR_SPEED
        elif self.direction == 'right':
            self.rect.x += CAR_SPEED

        # Verificar colisión con otros carros
        for carro in carros:
            if carro != self and self.rect.colliderect(carro.rect):
                self.has_collision = True
                if self.direction == 'up':
                    self.rect.y = carro.rect.y - self.rect.height
                elif self.direction == 'down':
                    self.rect.y = carro.rect.y + carro.rect.height
                elif self.direction == 'left':
                    self.rect.x = carro.rect.x - self.rect.width
                elif self.direction == 'right':
                    self.rect.x = carro.rect.x + carro.rect.width
                break
        else:
            self.has_collision = False

    def draw(self, screen):
        pygame.draw.rect(screen, self.color, self.rect)


# Clase Semaforo
class Semaforo:
    def __init__(self, x, y, size):
        self.light_color = RED
        self.collider_color = BLUE
        self.light_rect = pygame.Rect(x, y, size, size)
        self.collider_rect = pygame.Rect(x - size, y - size, size * 3, size * 3)

    def change_color(self):
        if self.light_color == RED:
            self.light_color = GREEN
        else:
            self.light_color = RED

    def is_red(self):
        return self.light_color == RED

    def draw(self, screen):
        pygame.draw.rect(screen, self.collider_color, self.collider_rect)
        pygame.draw.rect(screen, self.light_color, self.light_rect)


# Crear objetos
carros = []
carros.append(Carro(WIDTH // 2 - 20, HEIGHT - 80, 40, 20, WHITE, 'up'))
carros.append(Carro(WIDTH // 2 - 20, HEIGHT - 120, 40, 20, WHITE, 'up'))
carros.append(Carro(WIDTH // 2 - 20, HEIGHT - 160, 40, 20, WHITE, 'up'))

semaforo = Semaforo(WIDTH // 2 - 20, HEIGHT // 2 - 20, 40)

# Bucle principal del juego
running = True
while running:
    # Manejo de eventos
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Lógica del juego
    for carro in carros:
        carro.move(semaforo, carros)
        if carro.rect.colliderect(semaforo.collider_rect):
            carro.has_collision = True
        else:
            carro.has_collision = False
            
    # Cambiar color del semáforo cada segundo
    if pygame.time.get_ticks() % 1000 == 0:
        semaforo.change_color()

    # Dibujar en la pantalla
    screen.fill((0, 0, 0))  # Limpiar pantalla
    for carro in carros:
        carro.draw(screen)
    semaforo.draw(screen)
    pygame.display.flip()  # Actualizar pantalla

    clock.tick(FPS)

# Salir del juego
pygame.quit()


# Multiagentes con formato propuesta

In [7]:
import random

# Clase principal para la simulación
class TransportSimulation:
    def __init__(self):
        self.passengers = []
        self.drivers = []
        self.vehicles = []
        self.stations = []

    def add_passenger(self, name, origin, destination):
        self.passengers.append(Passenger(name, origin, destination))

    def add_driver(self, name):
        self.drivers.append(Driver(name))

    def add_vehicle(self, name, current_station):
        self.vehicles.append(Vehicle(name, current_station))

    def add_station(self, name):
        self.stations.append(Station(name))

    def simulate(self, num_steps):
        for step in range(num_steps):
            print("Paso de tiempo:", step)

            # Movimiento de los vehículos
            for vehicle in self.vehicles:
                vehicle.move(self.stations)

            # Actualizar estado de los pasajeros
            for passenger in self.passengers:
                passenger.update(self.stations, self.drivers)

            # Mostrar información
            for passenger in self.passengers:
                print(passenger.name, "- Estado:", passenger.status)

            print("----------------------")


# Clase para el agente pasajero
class Passenger:
    def __init__(self, name, origin, destination):
        self.name = name
        self.origin = origin
        self.destination = destination
        self.status = "Esperando"

    def update(self, stations, drivers):
        if self.status == "Esperando":
            for station in stations:
                if station.name == self.origin:
                    available_drivers = self.find_available_drivers(drivers)
                    if available_drivers:
                        driver = random.choice(available_drivers)
                        self.status = "En ruta"
                        print(self.name, "ha solicitado un viaje a", self.destination, "al conductor", driver.name)
                        driver.assign_passenger(self)
                    else:
                        print("No hay conductores disponibles en la estación", self.origin, "para", self.name)
                    break
        elif self.status == "En ruta":
            for station in stations:
                if station.name == self.destination:
                    self.status = "Llegado"
                    print(self.name, "ha llegado a la estación", self.destination)
                    break

    def find_available_drivers(self, drivers):
        available_drivers = []
        for driver in drivers:
            if not driver.has_assigned_passenger():
                available_drivers.append(driver)
        return available_drivers


# Clase para el agente conductor
class Driver:
    def __init__(self, name):
        self.name = name
        self.assigned_passenger = None

    def has_assigned_passenger(self):
        return self.assigned_passenger is not None

    def assign_passenger(self, passenger):
        self.assigned_passenger = passenger


# Clase para el agente vehículo
class Vehicle:
    def __init__(self, name, current_station):
        self.name = name
        self.current_station = current_station

    def move(self, stations):
        for station in stations:
            if station.name == self.current_station:
                index = stations.index(station)
                if index < len(stations) - 1:
                    self.current_station = stations[index + 1].name
                    print(self.name, "se ha movido a la estación", self.current_station)
                    break


# Clase para la estación
class Station:
    def __init__(self, name):
        self.name = name


# Crear una instancia de la simulación
simulation = TransportSimulation()

# Agregar pasajeros
simulation.add_passenger("Pasajero1", "EstaciónA", "EstaciónB")
simulation.add_passenger("Pasajero2", "EstaciónB", "EstaciónC")

# Agregar conductores y vehículos
simulation.add_driver("Conductor1")
simulation.add_driver("Conductor2")
simulation.add_vehicle("Vehículo1", "EstaciónA")
simulation.add_vehicle("Vehículo2", "EstaciónB")

# Agregar estaciones
simulation.add_station("EstaciónA")
simulation.add_station("EstaciónB")
simulation.add_station("EstaciónC")

# Ejecutar la simulación durante 10 pasos de tiempo
simulation.simulate(10)


Paso de tiempo: 0
Vehículo1 se ha movido a la estación EstaciónB
Vehículo2 se ha movido a la estación EstaciónC
Pasajero1 ha solicitado un viaje a EstaciónB al conductor Conductor1
Pasajero2 ha solicitado un viaje a EstaciónC al conductor Conductor2
Pasajero1 - Estado: En ruta
Pasajero2 - Estado: En ruta
----------------------
Paso de tiempo: 1
Vehículo1 se ha movido a la estación EstaciónC
Pasajero1 ha llegado a la estación EstaciónB
Pasajero2 ha llegado a la estación EstaciónC
Pasajero1 - Estado: Llegado
Pasajero2 - Estado: Llegado
----------------------
Paso de tiempo: 2
Pasajero1 - Estado: Llegado
Pasajero2 - Estado: Llegado
----------------------
Paso de tiempo: 3
Pasajero1 - Estado: Llegado
Pasajero2 - Estado: Llegado
----------------------
Paso de tiempo: 4
Pasajero1 - Estado: Llegado
Pasajero2 - Estado: Llegado
----------------------
Paso de tiempo: 5
Pasajero1 - Estado: Llegado
Pasajero2 - Estado: Llegado
----------------------
Paso de tiempo: 6
Pasajero1 - Estado: Llegado
Pas